In [756]:
!pip install pandas
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import tree

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [757]:
li = []
with open('adult.data') as f:
    lines = f.readlines()
    for line in lines:
        li.append(line[:-1].split(", "))

df = pd.DataFrame(li)

In [758]:
# rename columns
df = df.rename(columns={0: "age", 1: "workclass", 2: "fnlwgt", 3: "education", 4: "education-num", 5: "marital-status", 6: "occupation", 7: "relationship", 8: "race", 9: "sex", 10: "capital-gain", 11: "capital-loss", 12: "hours-per-week", 13: "native-country", 14: "income"})

In [759]:
# drop rows that has missing value 
df = df.dropna()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [760]:
# drop missing values
label_workclass = ['Private', 'Self-emp-not-inc', 'Self-emp-inc', 'Federal-gov', 'Local-gov', 'State-gov', 'Without-pay', 'Never-worked']
label_occupation = ['Tech-support', 'Craft-repair', 'Other-service', 'Sales', 'Exec-managerial', 'Prof-specialty', 'Handlers-cleaners', 'Machine-op-inspct', 'Adm-clerical', 'Farming-fishing', 'Transport-moving', 'Priv-house-serv', 'Protective-serv', 'Armed-Forces']
label_native_country = ['United-States', 'Cambodia', 'England', 'Puerto-Rico', 'Canada', 'Germany', 'Outlying-US(Guam-USVI-etc)', 'India', 'Japan', 'Greece', 'South', 'China', 'Cuba', 'Iran', 'Honduras', 'Philippines', 'Italy', 'Poland', 'Jamaica', 'Vietnam', 'Mexico', 'Portugal', 'Ireland', 'France', 'Dominican-Republic', 'Laos', 'Ecuador', 'Taiwan', 'Haiti', 'Columbia', 'Hungary', 'Guatemala', 'Nicaragua', 'Scotland', 'Thailand', 'Yugoslavia', 'El-Salvador', 'Trinadad&Tobago', 'Peru', 'Hong', 'Holand-Netherlands']
unknown_index = []

for i in range(len(df['workclass'])):
    if df['workclass'][i] not in label_workclass:
        unknown_index.append(i)
print("Missing value cumulation with workclass: ", len(unknown_index))


for i in range(len(df['occupation'])):
    if df['occupation'][i] not in label_occupation:
        unknown_index.append(i)
print("Missing value cumulation with occupation: ", len(unknown_index))


for i in range(len(df['native-country'])):
    if df['native-country'][i] not in label_native_country:
        unknown_index.append(i)        
print("Missing value cumulation with native-country: ", len(unknown_index))
df = df.drop(set(unknown_index))

Missing value cumulation with workclass:  1836
Missing value cumulation with occupation:  3679
Missing value cumulation with native-country:  4262


In [761]:
# drop meaningless column
df = df.drop(columns=['fnlwgt', 'capital-gain', 'capital-loss', 'native-country'])

In [762]:
# one-hot encoding on nominal value columns
df = pd.get_dummies(df, dtype=int, columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex'])

In [763]:
# Binarization of income column
df = df.replace({'<=50K': 0, ">50K": 1})

In [764]:
# split data into training and test data
x = df[::].drop(columns=['income'])
y = df[:]['income']
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2)

In [765]:
# training DTs using training data
dt = tree.DecisionTreeClassifier()
dt = clf.fit(train_x, train_y)

In [766]:
print('Training Accuracy: %0.4f' % dt.score(train_x, train_y))

Training Accuracy: 0.9668


In [767]:
print('Test Accuracy: %0.4f' % dt.score(test_x, test_y))

Test Accuracy: 0.7845


In [768]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(dt, x, y, cv=10)
print('Cross Validatoin Accuracy: %0.4f' % (scores.mean()))

Cross Validatoin Accuracy: 0.7807
